In [1]:
import sys
import pathlib
solitaire_path = pathlib.Path('../')

if solitaire_path not in sys.path:
    sys.path.append(str(solitaire_path))

import os, random, math, json, functools, itertools, base64, importlib
# import tqdm

from collections import defaultdict
from datetime import datetime, timedelta, date
from google.protobuf.json_format import MessageToDict
import numpy as np
import pandas as pd
    
from sklearn.model_selection import train_test_split
    
from solitaire_core.game import *
from solitaire_core import game, text_renderer
from solitaire_ai import vectorization

In [2]:
grs = []
with open("../2019-05-30-08-28-35-win-game-records.b64") as f:
    line = f.readline()
    while line:
        gr = GameRecord()
        gr.MergeFromString(base64.b64decode(line))
        grs.append(gr)
        
        line = f.readline()

print(len(grs))
        
game_states = []
actions = []
for gr in grs:
    gs = VisibleGameState()
    hgs = HiddenGameState()

    gs.MergeFrom(gr.initial_state)
    hgs.MergeFrom(gr.initial_hidden_state)

    for a in gr.actions:
        game_states.append(vectorization.game_state_to_array(gs))
        actions.append(vectorization.action_to_onehot(a))

        res = game._try_apply_action(gs, hgs, a)
        assert is_valid_game_state(gs)

        if not res:
            raise Exception()

assert len(game_states) == len(actions)

print(len(game_states))
print("Done")

236
33699
Done


In [7]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(game_states, dtype="float")
labels = np.array(actions)

(train_x, test_x, train_y, test_y) = train_test_split(data, labels, test_size=0.25)